In [ ]:
import os
import sys
import subprocess
import json
import textwrap
import pathlib

REPO_URL = "https://github.com/willloe/MSML640_Group10.git"
REPO_DIR = "/content/MSML640_Group10"
BRANCH = "feature/hires-upscale-inpaint"

if not pathlib.Path(REPO_DIR).exists():
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
else:
    print("Repo already present; pulling latest...")
    subprocess.run(["git", "-C", REPO_DIR, "pull", "--ff-only"], check=True)

subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin", BRANCH], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", BRANCH], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull", "--ff-only"], check=True)

SRC = f"{REPO_DIR}/packages/diffusion/src"
if SRC not in sys.path:
    sys.path.append(SRC)

import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

print("\nSmoke_sdxl_load")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_sdxl_load.py"], check=False)

print("\nSmoke_synthetic")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_synthetic.py"], check=False)

print("\nSmoke_infer (base)")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_infer.py"], check=False)

print("\nSmoke_infer (controlnet edge, dpmpp)")
subprocess.run([
    "python", f"{REPO_DIR}/scripts/smoke_infer.py",
    "--use_controlnet", "1",
    "--control_from", "edge",
    "--scheduler", "dpmpp",
    "--steps", "14",
    "--guidance", "6.0",
    "--width", "1024",
    "--height", "768",
    "--seed", "1234",
], check=False)

print("\nSmoke_upscale_inpaint")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_upscale_inpaint.py"], check=False)

print("\nCompleted. Check /content/MSML640_Group10/outputs for images.")